## Smodeling 

https://habr.com/ru/companies/ru_mts/articles/485980/
https://causalml.readthedocs.io/en/latest/methodology.html#s-learner https://github.com/uber/causalml/tree/master/causalml/inference/meta

In [1]:
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(Path('/Users/ogrobertino/UpliftModelingResearch/').resolve()))
%load_ext autoreload
%autoreload 2

In [2]:
parameters = {
    # 'iterations': (20, 100),
    'iterations': (1, 2),
    'learning_rate': (0.01, 0.5),
    'depth': (4, 15)
}

In [3]:
from src.utils import get_paths_train_test
from src.factory import SModelFactory

Failed to import duecredit due to No module named 'duecredit'


In [4]:
def train_test_model(ds_name, features_percent, factory, config, batch_size=32, max_size=100000):
    train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)
    model, train, test = factory.create(config, train_path, test_path)
    model.fit(train)
    predicted = model.predict(test)
    write(model, test, predicted, ds_name=ds_name, features_percent=features_percent,
          batch_size=batch_size, max_size=max_size)  

from src.utils import get_paths_train_test, train_test_model
from src.factory import SModelFactory
from src.configs_generation import generate_random_configs_smodel
from tqdm import tqdm

configs = generate_random_configs_smodel(parameters, count=10)

In [78]:
!pip index versions catboost

catboost (1.2.7)
Available versions: 1.2.7, 1.2.6, 1.2.5, 1.2.3, 1.2.2, 1.2.1.1, 1.2.1, 1.2
  INSTALLED: 1.2.7
  LATEST:    1.2.7

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install catboost==1.2.7 -q


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
ds_name = 'lazada'
features_percent = 50
factory = SModelFactory
config = configs[0]
batch_size=32
max_size=100000
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)
model, train, test = factory.create(config, train_path, test_path)

In [6]:
model.fit(train)

In [7]:
model.predict(test)

ValueError: assignment destination is read-only

In [23]:
model.model.models[1].feature_importances_

array([1.16718187, 3.49240277, 3.8610285 , 1.86314936, 5.71133975,
       1.41689097, 3.77339136, 3.77306756, 6.47462965, 4.29905668,
       5.51837687, 4.87652603, 3.35103278, 6.28659115, 1.816789  ,
       3.26879797, 3.01199695, 3.22538547, 5.18187544, 1.21125904,
       4.30864201, 1.83671831, 0.14758285, 0.0751762 , 0.04351329,
       3.97226563, 0.14227934, 0.09338793, 0.20847555, 0.87286104,
       0.53561725, 0.01019148, 0.0293831 , 1.13024044, 0.07910104,
       0.06264504, 1.20548069, 0.        , 0.        , 0.02297012,
       4.13422867, 2.66086161, 4.84760925])

In [8]:
import pandas as pd
import numpy as np

from packaging import version
from xgboost import __version__ as xgboost_version


def convert_pd_to_np(*args):
    output = [obj.to_numpy() if hasattr(obj, "to_numpy") else obj for obj in args]
    return output if len(output) > 1 else output[0]

In [20]:
X = test.data.copy(deep=True)

In [66]:
X_new.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [77]:
scores = test.data.copy(deep=True)
scores['score'] = model.model.predict(scores.loc[:, test.cols_features])

ValueError: assignment destination is read-only

In [49]:
model.model.models[1].best_score_

{'learn': {'Logloss': 0.5598454431903296}}

In [70]:
X_new = np.hstack((np.zeros((X.shape[0], 1)), X))
# kek = model.model.models[1].predict(X_new)

# set the treatment column to one (the treatment group)
# X_new[:, 0] = 1
# yhat_ts[group] = model.predict(X_new)

In [68]:
X_new.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [72]:
test.data.to_numpy().flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [75]:
kek = model.model.models[1].predict(test.data.loc[:, test.cols_features])
test.data.to_numpy().flags

CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:72: Feature 42 is present in model but not in pool.

In [ ]:
.loc[:, test.cols_features]

In [69]:
kek = model.model.models[1].predict(X_new)
X_new.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : False
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [62]:
np.array??

Docstring:
array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
      like=None)

Create an array.

Parameters
----------
object : array_like
    An array, any object exposing the array interface, an object whose
    ``__array__`` method returns an array, or any (nested) sequence.
    If object is a scalar, a 0-dimensional array containing object is
    returned.
dtype : data-type, optional
    The desired data-type for the array. If not given, NumPy will try to use
    a default ``dtype`` that can represent the values (by applying promotion
    rules when necessary.)
copy : bool, optional
    If true (default), then the object is copied.  Otherwise, a copy will
    only be made if ``__array__`` returns a copy, if obj is a nested
    sequence, or if a copy is needed to satisfy any of the other
    requirements (``dtype``, ``order``, etc.).
order : {'K', 'A', 'C', 'F'}, optional
    Specify the memory layout of the array. If object is not an array, the
    newly crea

In [61]:
X_new

array([[ 0.00000000e+00,  2.00000000e+00,  6.00000000e+00, ...,
        -1.67841480e-02,  1.36126210e-02,  4.36309862e+00],
       [ 0.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
        -1.67841480e-02,  1.36126210e-02,  5.23909801e+00],
       [ 0.00000000e+00,  2.00000000e+00,  4.40000000e+01, ...,
        -1.67841480e-02,  1.36126210e-02,  1.25276297e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  3.65000000e+02, ...,
         4.06665200e-03,  2.07665160e-01,  0.00000000e+00],
       [ 0.00000000e+00,  2.00000000e+00,  3.65000000e+02, ...,
         4.06665200e-03,  2.07665160e-01,  1.38629436e+00],
       [ 0.00000000e+00,  1.00000000e+00,  5.80000000e+01, ...,
         9.19378200e-04,  1.27585020e-03,  2.70805020e+00]])

In [59]:
X_new[:, 0] = 1

ValueError: assignment destination is read-only

In [51]:
kek

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
X_new[:, 0] = 1
# yhat_ts[group] = model.predict(X_new)

ValueError: assignment destination is read-only

In [53]:
X_new.shape

(181669, 43)

In [27]:
X = scores.loc[:, test.cols_features]

In [28]:
treatment = None
y = None
X, treatment, y = convert_pd_to_np(X, treatment, y)
yhat_cs = {}
yhat_ts = {}

In [30]:
X_new = np.hstack((np.zeros((X.shape[0], 1)), X))

In [31]:
X_new

array([[ 0.00000000e+00,  2.00000000e+00,  6.00000000e+00, ...,
        -1.67841480e-02,  1.36126210e-02,  4.36309862e+00],
       [ 0.00000000e+00,  2.00000000e+00,  2.00000000e+00, ...,
        -1.67841480e-02,  1.36126210e-02,  5.23909801e+00],
       [ 0.00000000e+00,  2.00000000e+00,  4.40000000e+01, ...,
        -1.67841480e-02,  1.36126210e-02,  1.25276297e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  3.65000000e+02, ...,
         4.06665200e-03,  2.07665160e-01,  0.00000000e+00],
       [ 0.00000000e+00,  2.00000000e+00,  3.65000000e+02, ...,
         4.06665200e-03,  2.07665160e-01,  1.38629436e+00],
       [ 0.00000000e+00,  1.00000000e+00,  5.80000000e+01, ...,
         9.19378200e-04,  1.27585020e-03,  2.70805020e+00]])

In [32]:
X.shape, X_new.shape

((181669, 42), (181669, 43))

In [35]:
X_new[:, 0] = 1

In [36]:
X_new.shape

(181669, 43)

In [19]:
model.predict(test)

ValueError: assignment destination is read-only

In [ ]:
def train_test_model(ds_name, features_percent, factory, config, batch_size=32, max_size=100000):
    train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)
    model, train, test = factory.create(config, train_path, test_path)
    model.fit(train)
    predicted = model.predict(test)
    write(model, test, predicted, ds_name=ds_name, features_percent=features_percent,
          batch_size=batch_size, max_size=max_size)  

In [8]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import SModelFactory
from src.configs_generation import generate_random_configs_smodel
from tqdm import tqdm

configs = generate_random_configs_smodel(parameters, count=10)

for i in tqdm(range(len(configs)), desc="Training models"):
    train_test_model(ds_name='lazada', features_percent=100, factory=SModelFactory, config=configs[i])    

Training models:   0%|                                                                   | 0/10 [00:34<?, ?it/s]


ValueError: assignment destination is read-only

In [6]:
from src.global_params import BASE_PATH
import pandas as pd
pd.read_csv(BASE_PATH / "exps/stats.tsv", sep='\t')

,Model,Path,Dataset,Features Percent,Latency (ms),Binary Size (MB),AUUC (test),Precision@5,Precision@10,Precision@15,Precision@20,Precision@25,Precision@50,Compressions
0,TModel,exps/lazada/100/0,lazada,100,0.059281,3.112857,0.536919,0.009425,0.009378,0.007088,0.009378,0.006507,0.004036,{}
1,TModel,exps/lazada/100/1,lazada,100,0.056143,25.315657,0.701555,0.001197,0.008309,0.008484,0.008309,0.007162,0.005920,{}
2,TModel,exps/lazada/100/2,lazada,100,0.057361,29.465457,0.695790,0.009794,0.010686,0.008920,0.010686,0.009760,0.005464,{}
3,TModel,exps/lazada/100/3,lazada,100,0.057964,4.878569,0.656579,0.011354,0.010685,0.009732,0.010685,0.008134,0.005860,{}
4,TModel,exps/lazada/100/4,lazada,100,0.056108,0.150800,0.478659,0.016890,0.009386,0.007613,0.009386,0.005876,0.003602,{}
5,TModel,exps/lazada/100/5,lazada,100,0.057290,19.446937,0.571644,-0.004150,0.007774,0.006016,0.007774,0.005597,0.004509,{}
6,TModel,exps/lazada/100/6,lazada,100,0.058309,17.388096,0.449403,0.010904,0.005254,0.003474,0.005254,0.004186,0.004041,{}
7,TModel,exps/lazada/100/7,lazada,100,0.055492,1.324496,0.621997,0.011158,0.008495,0.008291,0.008495,0.008107,0.005221,{}
8,TModel,exps/lazada/100/8,lazada,100,0.057467,12.526577,0.491589,-0.001493,0.003126,0.004580,0.003126,0.006341,0.004382,{}
9,TModel,exps/lazada/100/9,lazada,100,0.057034,0.842080,0.685294,0.018461,0.011393,0.009931,0.011393,0.008308,0.005581,{}


In [7]:
from src.models.ICausalML.Models import TModel
path = pd.read_csv(BASE_PATH / "exps/stats.tsv", sep='\t').Path.iloc[-1]
model = TModel(from_load=True, path = str(BASE_PATH) + "/" + path)

Model loaded from /Users/ogrobertino/UpliftModelingResearch/exps/lazada/100/9/model.pkl.
Config loaded from /Users/ogrobertino/UpliftModelingResearch/exps/lazada/100/9/config.json.
